In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai2.vision.all import *
from efficientnet_pytorch import EfficientNet

In [3]:
np.random.seed(1773)

In [4]:
CSV_PATH = "/home/ubuntu/auto/dataset/train.csv"
df = pd.read_csv(CSV_PATH)

In [5]:
exp_time = str(datetime.now().replace(second=0, microsecond=0)).replace(" ", "_")
SIZE = 255
BS = 64
GPU = 0
EXP_NAME = "{}-{}-{}".format(BS, SIZE, exp_time)
print(EXP_NAME)

64-255-2020-04-04_16:16:00


In [6]:
item_tfms = [Resize(SIZE)]

In [7]:
dls = ImageDataLoaders.from_df(df=df, path="/home/ubuntu/auto/dataset/train", 
                                item_tfms=item_tfms, valid_pct=2.0,
                                bs=BS, num_workers=16, device=GPU)

IndexError: single positional indexer is out-of-bounds

In [ ]:
dls.show_batch(max_n=9, figsize=(7,6))

In [ ]:
print(dls.vocab)
len(dls.vocab),dls.c

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=4)
learn = Learner(dls, model, metrics=[error_rate, accuracy], opt_func=ranger, cbs= MixUp())

In [ ]:
learn.fine_tune(20)

In [ ]:
learn.freeze()
learn.fit_sgdr(2,5, 1e-3)

In [ ]:
learn.unfreeze()
learn.fit_sgdr(2,5, slice(2e-7, 1e-4))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(7,6), dpi=60)
interp.most_confused(min_val=10)

In [ ]:
path = "/home/ubuntu/auto/dataset/test"
imgs = get_image_files(path)
test_dl = dls.test_dl(imgs)

In [ ]:
preds, _ = learn.get_preds(dl=test_dl)

In [ ]:
ids = [i for i in os.listdir(path)]
submission = pd.DataFrame({'Image': ids, 'Attire': preds[:,0], 'Decorationandsignage': preds[:,1], 'Food': preds[:,2], 'misc': preds[:,2]})

In [ ]:
for i, c in enumerate(dls.vocab):
    submission[c] = preds[:,i]

In [ ]:
sub = submission.drop("Image", axis=1)

In [ ]:
id_class = sub.idxmax(axis=1).tolist()

In [ ]:
submission = pd.DataFrame({'Image': ids, 'Class': id_class})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('/home/ubuntu/auto/submission/{}.csv'.format(EXP_NAME), index=False)